In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,375 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,125 kB]
Get:13 https://developer.download.nvidia.com/compute/

In [ ]:
import pandas as pd



import pyspark
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf

In [ ]:
# Load the data- update to load data using pyspark or SQL
df = pd.read_csv('/content/Amazon_Product_Reviews.csv')


FileNotFoundError: [Errno 2] No such file or directory: '/content/Amazon_Product_Reviews.csv'

In [ ]:
df.head()

In [ ]:
#Clean model
# Drop duplicates
df.dropna(subset=['Review_Text'], inplace=True)

# Text Cleaning
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

df['cleaned_review'] = df['Review_Text'].apply(preprocess)


In [35]:
df.head()

,Product ID,Product Name,Asins,Categories,Review Date,Recommended,Reviews_Helpful,Reviews_Ratings,Review_Text,Review_Title,cleaned_review,sentiment,sentiment_num
0,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Electronics,2017-09-03T00:00:00.000Z,False,0,3,I thought it would be as big as small paper bu...,Too small,thought would big small paper turn like palm t...,neutral,0
1,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Electronics,2017-06-06T00:00:00.000Z,True,0,5,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,kindle light easy use especially beach,positive,1
2,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Electronics,2018-04-20T00:00:00.000Z,True,0,4,Didnt know how much i'd use a kindle so went f...,Great for the price,didnt know much use kindle went lower end im h...,positive,1
3,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Electronics,2017-11-02T17:33:31.000Z,True,3,5,I am 100 happy with my purchase. I caught it o...,A Great Buy,100 happy purchase caught sale really good pri...,positive,1
4,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Electronics,2018-04-24T00:00:00.000Z,True,0,5,Solid entry level Kindle. Great for kids. Gift...,Solid entry-level Kindle. Great for kids,solid entry level kindle great kid gifted kid ...,positive,1


In [ ]:
#Normalize model- FROM CLASS NOTES

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df["cleaned_review"]).toarray()

In [ ]:
def assign_sentiment(rating):
    if rating >= 4:
        return 'positive'
    elif rating == 3:
        return 'neutral'
    else:
        return 'negative'
df['sentiment'] = df['Reviews_Ratings'].apply(assign_sentiment)
df['sentiment_num'] = df["sentiment"].map({'positive':1, 'negative':2, 'neutral':0 }).values

In [ ]:
df.head()

,Product ID,Product Name,Asins,Categories,Review Date,Recommended,Reviews_Helpful,Reviews_Ratings,Review_Text,Review_Title,cleaned_review,sentiment,sentiment_num
0,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Electronics,2017-09-03T00:00:00.000Z,False,0,3,I thought it would be as big as small paper bu...,Too small,thought would big small paper turn like palm t...,neutral,0
1,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Electronics,2017-06-06T00:00:00.000Z,True,0,5,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,kindle light easy use especially beach,positive,1
2,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Electronics,2018-04-20T00:00:00.000Z,True,0,4,Didnt know how much i'd use a kindle so went f...,Great for the price,didnt know much use kindle went lower end im h...,positive,1
3,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Electronics,2017-11-02T17:33:31.000Z,True,3,5,I am 100 happy with my purchase. I caught it o...,A Great Buy,100 happy purchase caught sale really good pri...,positive,1
4,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Electronics,2018-04-24T00:00:00.000Z,True,0,5,Solid entry level Kindle. Great for kids. Gift...,Solid entry-level Kindle. Great for kids,solid entry level kindle great kid gifted kid ...,positive,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Product ID       5000 non-null   object
 1   Product Name     5000 non-null   object
 2   Asins            5000 non-null   object
 3   Categories       5000 non-null   object
 4   Review Date      5000 non-null   object
 5   Recommended      5000 non-null   bool  
 6   Reviews_Helpful  5000 non-null   int64 
 7   Reviews_Ratings  5000 non-null   int64 
 8   Review_Text      5000 non-null   object
 9   Review_Title     4987 non-null   object
 10  cleaned_review   5000 non-null   object
 11  sentiment        5000 non-null   object
 12  sentiment_num    5000 non-null   int64 
dtypes: bool(1), int64(3), object(9)
memory usage: 473.8+ KB


In [33]:
x_cat=df.dtypes[df.dtypes=="object"].index.tolist()
x_cat

['Product ID',
 'Product Name',
 'Asins',
 'Categories',
 'Review Date',
 'Review_Text',
 'Review_Title',
 'cleaned_review',
 'sentiment']

In [34]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_arr = enc.fit_transform(df[x_cat])

# Add the encoded variable names to the dataframe
encode_df = pd.DataFrame(encode_arr, columns=enc.get_feature_names_out(x_cat))
encode_df.head()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Product ID_AV-XeQLWuC1rwyj_gbP5,Product ID_AVpfIfGA1cnluZ0-emyp,Product ID_AVpfpK8KLJeJML43BCuD,Product ID_AVpftoij1cnluZ0-p5n2,Product ID_AVpgdkC8ilAPnD_xsvyi,Product ID_AVph0EeEilAPnD_x9myq,Product ID_AVphPmHuilAPnD_x3E5h,Product ID_AVpidLjVilAPnD_xEVpI,Product ID_AVpjEN4jLJeJML43rpUe,Product ID_AVqVGWLKnnc1JgDc3jF1,...,cleaned_review_äôve always loved amazon alexa smart product use control smart home setup,cleaned_review_äôve amazon alexa since first hit street two year ago echo show awesome much alexa echo dot love,cleaned_review_äôve asked itquestions answer unknown could help,cleaned_review_äôve enjoy echo since purchased light bulb great buying product,cleaned_review_äôve enjoying item getting information fast best purchase ever made,cleaned_review_äôve hesitant purchase item äôt feel actual need alexa changed mind use daily timer turning appliance checking weather adjusting heat alexa even arm alarm system,cleaned_review_äôve using amazon echo day released market unfortunately volume control wheel broke quickly went local best buy purchased next generation amazing improvement concerning device really truly love alexa äôt know would without,sentiment_negative,sentiment_neutral,sentiment_positive
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
# Split the data into features
y = df["sentiment"].values
X = df.drop(["sentiment"],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
#Standardize model- FROM CLASS NOTES
scaler = StandardScaler()

# Fit and transform the training data
X_trained_scaled = scaler.fit_transform(X_train)

# Scale the data
#X_train_scaled = X.transform(X_train)
X_test_scaled = X.transform(X_test)

ValueError: could not convert string to float: 'AVpjEN4jLJeJML43rpUe'

In [ ]:
#Initialize model

In [ ]:
# Compile the model- FROM CLASS NOTES
df.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
#Train model- FROM CLASS NOTES
fit_model = df.fit(x_train_scaled,y_train,epochs=100)

In [ ]:
#Evaluate model- FROM CLASS NOTES
#0.75% classification accuracy or 0.8 r-squared
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")